# Simple map visualization of traffic accident data in Helsinki 2018-2020

Datasets downloaded in July 2021 from National Land Survey of Finland (Maanmittauslaitos) web service Paikkatietoikkuna \
https://kartta.paikkatietoikkuna.fi/ \
Datasets are owned by Statistics Finland (Tilastokeskus) and are released under CC BY 4.0 Licence \
https://tilastokeskus.fi/org/lainsaadanto/copyright_en.html \
https://creativecommons.org/licenses/by/4.0/deed.fi \
Statistics Finland received the data from the Finnish Police. \
For this analysis the whole capital region data was not used and some parts of Espoo and Vantaa are missing.

## Importing

Let's install folium for map visualization and pyproj for converting coordinates to a form that folium can understand. Let's also import pandas for dataset manipulation.

In [1]:
pip install folium pyproj

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from pyproj import CRS, Transformer
import folium

Let's read the pre-downloaded datasets with pandas

In [3]:
liikenne = pd.read_csv('data/tieliikenneonnettomuudet2020.csv')
liikenne2 = pd.read_csv('data/tieliikenneonnettomuudet2019.csv')
liikenne3 = pd.read_csv('data/tieliikenneonnettomuudet2018.csv')

## Exploration and manipulation

In [4]:
liikenne.head()

,kello,onntyyppi,lkmpp,lkmjk,lkmmuukulk,lkmhapa,lkmlaka,lkmmo,lkmmp,x,vvonn,kkonn,vakav,y
0,15.00-15.59,2,2,0,0,0,0,0,0,385275.65,2020,8,2,6674644.92
1,09.00-09.59,0,0,0,0,1,0,0,1,385301.33,2020,4,3,6674717.44
2,14.00-14.59,8,0,0,0,1,0,0,0,384992.50,2020,4,2,6675802.79
3,07.00-07.59,8,0,0,0,1,0,0,0,385082.40,2020,9,2,6675823.05
4,15.00-15.59,0,0,0,0,2,0,0,0,385212.13,2020,3,2,6676236.73


Paikkatietoikkuna gives the following information about the different columns: <br>
vvonn = year of the accident <br>
kkonn = month of the accident <br>
kello = time of the accident <br>
vakav = seriousness of the accident: <br>
1 = accident resulting in death, <br>
2 = accident resulting in injury, <br>
3 = accident resulting in serious injury <br>
<br>
onntyyppi = type of accident: <br>
0 = same direction of travel (going straight), <br>
1 = same direction of travel (turning), <br>
2 = opposite direction of travel (going straight), <br>
3 = opposite direction of travel (turning), <br>
4 = intersecting direction of travel (going straight), <br>
5 = intersecting direction of travel (turning), <br>
6 = pedestrian accident (on pedestrian crossing), <br>
7 = pedestrian accident (elsewhere), <br>
8 = running off the road, <br>
9 = other accident<br>
<br>
lkmhapa = number of passenger cars and vans in the accident <br>
lkmlaka = number of buses and lorries in the accident <br>
lkmjk = number of pedestrians in the accident <br>
lkmpp = number of cyclists in the accident <br>
lkmmo = number of mopeds in the accident <br>
lkmmp = number of motor cycles in the accident <br>
lkmmuukulk = number of other vehicles in the accident <br>
<br>
x = x co-ordinate of the accident<br>
y = y co-ordinate of the accident <br>
<br>
For this analysis we want to visualize the seriousness of accidents at different locations, so we'll be looking at the x & y coordinates and the "vakav" column. <br>
<br>
Each of the datasets contains data for 1 year, so in order to use them together, let's first combine them into 1 dataset.

In [5]:
liikenne4 = pd.concat([liikenne, liikenne2, liikenne3], ignore_index=True)

## Visualization

The dataset uses a different type of coordinates than what Folium uses, so we have to use pyproj to change the coordinates to the regular format that Folium can understand. After that we can visualize the accidents on the map. <br>
<br>
In this case we'll show most serious accidents (resulting in death, vakav = 1) with red color and bigger circle (radius = 7), <br>
accidents with serious injuries (vakav = 3) with yellow and medium size circle (radius = 5) <br>
and the accidents with normal injuries (vakav = 2) with grey color and smallest circle (radius = 2).

In [6]:
incidents = folium.map.FeatureGroup()

m = folium.Map(location=[60.213228, 24.9558931], zoom_start=11)

crs_in = CRS.from_string("EPSG:3067")
crs_out = CRS.from_string("EPSG:4326")
transformer = Transformer.from_crs(crs_in, crs_out)

for lat, lng, vakav in zip(liikenne4.y, liikenne4.x, liikenne4.vakav):
    lat2, lng2 = transformer.transform(lng,lat)
    if vakav==1:
        color='red'
        radius=7
        opacity=1
        fill_opacity=1
    elif vakav == 3:
        color='yellow'
        radius=5
        opacity=1
        fill_opacity=1
    else:
        color='black'
        radius=2
        opacity=0.6
        fill_opacity=0.6
    incidents.add_child(
        folium.features.CircleMarker(
            [lat2, lng2],
            radius=radius,             
            color='black',
            fill=True,
            fill_color=color,
            opacity=opacity,
            fill_opacity=fill_opacity
        )
    )

# add incidents to map
m.add_child(incidents)

m

We can see that most accidents resulting in deaths happened close to bigger roads. The vast majority of accidents resulted in regular injuries and only a minority resulted in severe injuries or death. There are also many areas that haven't had any accidents at all.